# Start Ollama Server

1. Open a terminal

    ☰ > *Terminal* > *New Terminal*

2. Start the Ollama server 

    ```bash
    ollama serve
    ```

3. Do not close this terminal

4. Open another terminal
5. Pull the model *"qwen3:4b"* (if not already done)

    ```bash
    ollama pull qwen3:4b
    ```
5. Close this terminal



# Define Environment Variables

In [1]:
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen3:4b"
LLM_SEED = 42 
LLM_TEMPERATURE = 0.0
TEST_PROMPT0 = "What time is it?"
TEST_PROMPT1 = "What is the price of gold right now?" 
TEST_PROMPT3 = "How many 1 in 111111111111111?"

# Initialize Ollama Chatbot

In [2]:
from langchain_ollama import ChatOllama

# minimize randomness for reproducibility
seed = 42
temperature = 0

# Set up the Ollama chat model with specified LLM model and parameters
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    seed=LLM_SEED,
    stream=True
)

# Chatbot Class

For sake of simplicity, we define a chatbot class that uses the Ollama server to interact with the Qwen-3 model. This class will handle the initialization of the Ollama client, setting up the model, and providing a method to generate responses based on user input.

See file `src/chatbot.py` for details.

In [52]:
from src.Chatbot import Chatbot

## Zero-shot Prompting

Zero-shot prompting uses the ability of Large Language Models (LLMs) to recognize or make predictions about classes or tasks it has never seen during training. For example, you can ask an LLM to classify text as *positive*, *negative*, or *neutral* — a task known in Machine Learning as *sentiment analysis*.

In [3]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

instructions = "You are a Sentiment analyzer. Classify the text into neutral, negative, or positive sentiment. Only return the sentiment classification."
chatbot = Chatbot(llm=llm, history=[SystemMessage(content=instructions)])

chatbot.invoke("I love programming in Python!")  # Example prompt (positive sentiment)
chatbot.invoke("I hate bugs in my code!")  # Another example prompt (negative sentiment)
chatbot.invoke("The sky is blue.")  # Example prompt (neutral sentiment)

NameError: name 'Chatbot' is not defined

## Few-Shot Prompting

Zero-shot prompting uses the ability of Large Language Models (LLMs) to learn to perform a task with only a small number of labeled examples (shots) for each class, instead of needing large datasets. How it works:

- The model is given a few examples (like 2, 5, or 10) of each class.
- Using these examples, it generalizes to classify or predict new, unseen inputs.

In [ ]:
instructions = """
Example 1:  
Text: "Alice visited Paris last summer."  
Entities: Alice (Person), Paris (Location)

Example 2:  
Text: "Google was founded by Larry Page and Sergey Brin."  
Entities: Google (Organization), Larry Page (Person), Sergey Brin (Person)

Example 3:  
Text: "Amazon is headquartered in Seattle."  
Entities: Amazon (Organization), Seattle (Location)

Now identify entities (Organization, Person, location) in the following inputs.
"""

chatbot = Chatbot(llm=llm, history=[SystemMessage(content=instructions)])

chatbot.invoke("Microsoft acquired LinkedIn in 2016.")  # Example prompt
chatbot.invoke("Elon Musk is the CEO of SpaceX and Tesla.")  # Another example prompt
chatbot.invoke("The Eiffel Tower is located in Paris, France.")  # Example prompt

================================ System Message ================================


Example 1:  
Text: "Alice visited Paris last summer."  
Entities: Alice (Person), Paris (Location)

Example 2:  
Text: "Google was founded by Larry Page and Sergey Brin."  
Entities: Google (Organization), Larry Page (Person), Sergey Brin (Person)

Example 3:  
Text: "Amazon is headquartered in Seattle."  
Entities: Amazon (Organization), Seattle (Location)

Now identify entities (Organization, Person, location) in the following inputs.

================================ Human Message =================================

Microsoft acquired LinkedIn in 2016 in Los Angeles.
================================== Ai Message ==================================

<think>
Okay, let's see. The user provided a sentence: "Microsoft acquired LinkedIn in 2016 in Los Angeles." I need to identify the entities here. Let me break it down.

First, the main subjects and objects. "Microsoft" is a company, so that's an Organization

# Chain-of-Thought (CoT) Prompting

Chain-of-Thought (CoT) prompting is a technique that guides Large Language Models (LLMs) to break down complex problems into smaller, logical steps. By encouraging step-by-step reasoning, CoT helps the model arrive at more accurate and transparent solutions.

For example, instead of directly giving the answer to a math problem, the model is prompted to explain the reasoning process step-by-step before concluding (e.g., `"think step by step and provide the final answer at the end."`).

**Reference:**

[Wei et al., (2022). Wei, J., Wang, X., Schuurmans, D., Bosma, M., Xia, F., Chi, E., ... & Zhou, D. (2022). Chain-of-thought prompting elicits reasoning in large language models. *Advances in neural information processing systems*, 35, 24824-24837.](https://doi.org/10.48550/arXiv.2201.11903)

<div style="background-color:rgba(0, 0, 0, 0); padding: 10px; border: 1px solid rgba(255, 255, 255, 1); border-radius: 5px; width: 98%; margin: 10px auto;">
Recent LLMs like GPT-4 and LLaMA 3 now have built-in chain-of-thought capabilities as part of their cold start (see text between  &lt;think&gt; and &lt;/think&gt;). This means CoT reasoning happens automatically without special prompting, unless you explicitly want the model to show its intermediate steps in the final answer.
</div>

In [27]:
instructions = """You are a helpful assistant. Answer my questions in a helpful way."""
prompt = """
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the convenience store owner.
Then I went to buy 5 more apples and ate 1. How many apples do I have left?
"""
chatbot = Chatbot(llm=llm, history=[SystemMessage(content=instructions)])
chatbot.invoke(prompt)

================================ System Message ================================

You are a helpful assistant. Answer my questions in a helpful way.
================================ Human Message =================================


I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the convenience store owner.
Then I went to buy 5 more apples and ate 1. How many apples do I have left?

================================== Ai Message ==================================

<think>
Okay, let's see. The user bought 10 apples initially. Then they gave 2 to the neighbor and 2 to the convenience store owner. So first, I need to subtract those from the original 10. Let me calculate that: 10 minus 2 is 8, then minus another 2 would be 6. So after giving away those apples, they have 6 left.

Then they went to buy 5 more apples. So adding those to the 6 they had: 6 plus 5 equals 11. Now they have 11 apples. Then they ate 1, so subtract 1 from 11, which leaves them with

source: https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/chain-prompts#example-self-correcting-research-summary


In [45]:
from langchain_community.document_loaders import PyPDFLoader
import re

def sanitize(message: AIMessage) -> str:
    """Remove <think> tags and extra whitespace from the message content."""
    return re.sub(r"<think>.*?</think>", "", message.content, flags=re.DOTALL).strip()


instructions = "You are a helpful assistant. Answer my questions in a helpful way."
chatbot = Chatbot(llm=llm, history=[SystemMessage(content=instructions)])



### PROMPT 0: Summarize a medical research paper
RESEARCH_PAPER = "\n".join([page.page_content for page in PyPDFLoader("files/doudna2014crispr.pdf").lazy_load()])
prompt0 = f"""Summarize this medical research paper.

<paper>
{RESEARCH_PAPER}
</paper>

Focus on methodology, findings, and clinical implications.
"""
chatbot.clear_history()
SUMMARY = sanitize(chatbot.invoke(prompt0))

### PROMPT 1: Provide feedback on a research paper summary
prompt1 = f"""Your task is to provide feedback on a research paper summary. Here is a summary of a medical research paper:

<summary>
{SUMMARY}
</summary>

Here is the research paper:

<paper>
{RESEARCH_PAPER}
</paper>

Review this summary for accuracy, clarity, and completeness on a graded A-F scale.
"""
chatbot.clear_history()
FEEDBACK = sanitize(chatbot.invoke(prompt1))

### PROMPT 2: Improve the summary based on feedback
prompt2 = f"""Your task is to improve a paper summary given feedback. Here is the first draft of a medical research paper:
<summary>
{SUMMARY}
</summary>

Here is the research paper:
<paper>
{RESEARCH_PAPER}
</paper>

Here is the feedback:
<feedback>
{FEEDBACK}
</feedback>

Update the summary based on the feedback.
"""
chatbot.clear_history()
CORRECTED_SUMMARY = sanitize(chatbot.invoke(prompt2))


================================ Human Message =================================

Your task is to improve a paper summary given feedback. Here is the first draft of a medical research paper:
<summary>
**Focus on Methodology, Findings, and Clinical Implications**

---

### **Methodology**  
The paper outlines the CRISPR-Cas9 system, emphasizing its components and optimization strategies:  
1. **CRISPR-Cas9 Mechanism**:  
   - **Guide RNA (gRNA)**: Directs the Cas9 nuclease to target DNA sequences.  
   - **Cas9 Enzyme**: Creates double-strand breaks at the target site, enabling gene editing via homology-directed repair (HDR) or non-homologous end joining (NHEJ).  
   - **Variants**: Double-nicking (e.g., dCas9 with FokI), truncated gRNAs, and nickases (single-strand breaks) to enhance specificity.  

2. **Design and Analysis Tools**:  
   - **Computational Tools**: CHOPCHOP, GT-Scan, CasOT, and sgRNAcas9 for designing gRNAs, predicting off-target effects, and evaluating cleavage sites. 